In [47]:
import os
from pathlib import Path
os.chdir("/home/oralb/Masaüstü/End_to_end_classification_project/Chicken-Desease-Classification_Project/research")
%pwd

'/home/oralb/Masaüstü/End_to_end_classification_project/Chicken-Desease-Classification_Project/research'

In [54]:
os.chdir("../")
%pwd

'/home/oralb/Masaüstü/End_to_end_classification_project/Chicken-Desease-Classification_Project'

In [55]:
import tensorflow as tf

In [56]:
#model= tf.keras.models.load_model(filepath=str("/home/oralb/Masaüstü/End_to_end_classification_project/Chicken-Desease-Classification_Project/artifacts/training/model.h5"))


model = tf.keras.models.load_model("artifacts/training/model.h5")



In [57]:
from dataclasses import dataclass


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [58]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [64]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path= CONFIG_FILE_PATH,
        params_file_path= PARAMS_FILE_PATH):
        self.config= read_yaml(Path(str(config_file_path)))
        self.params= read_yaml(Path(params_file_path))
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/Chicken-fecal-images",
            all_params= self.params,
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config

In [65]:
from urllib.parse import urlparse

In [69]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [70]:
try:
    config= ConfigurationManager()
    val_config= config.get_evaluation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e


[2023-11-04 01:54:51,338: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-11-04 01:54:51,343: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-04 01:54:51,348: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


2023-11-04 01:54:52.363752: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-11-04 01:54:52.605604: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-11-04 01:54:53.677053: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2023-11-04 01:54:53.756001: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-11-04 01:54:54.216565: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


8/8 [==============================] - 58s 7s/step - loss: 0.3609 - accuracy: 0.9138
[2023-11-04 01:56:14,230: INFO: common: json file saved at: scores.json]
